# 法人・産業データの取得・可視化・分析

In [ ]:
# 環境変数とパス設定に用いるライブラリ
import os
from dotenv import load_dotenv
from pathlib import Path
# データ取得に用いるライブラリ
from io import StringIO
import requests
# データ操作に用いるライブラリ
import numpy as np
import pandas as pd
import openpyxl
# 可視化に用いるライブラリ
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

In [ ]:
from estat import (
    get_metainfo,
    get_statsdata,
    cleansing_statsdata,
    colname_to_japanese,
    create_hierarchy_dataframe,
)

In [ ]:
load_dotenv()
appId = os.getenv("ESTAT_APP_ID")
NTA_APP_ID = os.getenv("NTA_APP_ID")
api_token = os.getenv("gBizINFO_API_ACCESS_TOKEN")

In [ ]:
current_dir = Path.cwd()
data_path = (current_dir / "data" / "ch11").resolve()

## 法人・企業・会社のデータ

- [法人番号公表サイト](https://www.houjin-bangou.nta.go.jp/)
- [日本年金機構](https://www.nenkin.go.jp/index.html)
- [厚生年金・健康保険 適用事業所検索システム](https://www2.nenkin.go.jp/do/search_section/)
- [EDINET 閲覧サイト](https://disclosure2.edinet-fsa.go.jp/week0010.aspx)
- [gBizINFO](https://info.gbiz.go.jp/)
- [法人企業統計調査](https://www.mof.go.jp/pri/reference/ssc/index.htm)
- [経済センサス](https://www.stat.go.jp/data/e-census/index.html)
- [事業所母集団データベース研究会](https://www.stat.go.jp/info/kenkyu/jsdb/index.html)
- [経済構造実態調査](https://www.stat.go.jp/data/kkj/index.html)
- [個人企業経済調査](https://www.stat.go.jp/data/kojinke/index.html)


## 経済センサス

- [公表](https://www.stat.go.jp/data/kouhyou/e-stat_e-census2019.xml)
- [用語](https://www.stat.go.jp/data/e-census/2019/yougo.html)
- [中小企業基本法](https://www.chusho.meti.go.jp/soshiki/teigi.html)
- [賃金構造基本統計調査](https://www.mhlw.go.jp/toukei/itiran/roudou/chingin/kouzou/z2021/yougo.html)

### 産業・組織形態・雇用者規模別企業等数


In [ ]:
census_statsDataId = "0004006260"  # 令和3年経済センサス
census_meta = get_metainfo(appId, census_statsDataId)
census_metadata = census_meta["GET_META_INFO"]["METADATA_INF"]
census_total_num = census_metadata["TABLE_INF"]["OVERALL_TOTAL_NUMBER"]
census_total_num

In [ ]:
census_params = {"cdTab": "201-2021"}
census_data = get_statsdata(appId, census_statsDataId, params=census_params)
census_value = colname_to_japanese(cleansing_statsdata(census_data))
census_value.columns

In [ ]:
census_cond = census_value["企業産業中分類"] != "全産業（S_公務を除く）"
census_cond &= census_value["経営組織"] != "総数"
census_cond &= census_value["企業常用雇用者規模"] != "総数"
census_cols = [
    "企業産業中分類コード", "企業産業中分類",
    "経営組織コード", "経営組織",
    "企業常用雇用者規模コード", "企業常用雇用者規模",
    "値",
]
census_df = census_value.loc[census_cond, census_cols]
census_df.head()

In [ ]:
level_cat01 = create_hierarchy_dataframe(census_meta, 1)
level_cat01.head()

In [ ]:
level_cat02 = create_hierarchy_dataframe(census_meta, 2)
level_cat02

In [ ]:
lv01_df = level_cat01[["level4"] + [f"企業産業中分類階層{i}" for i in range(1, 5)]]
lv02_df = level_cat02[["level2", "経営組織階層1", "経営組織階層2"]].drop_duplicates()
corp_df = (census_df
    .merge(lv01_df, left_on="企業産業中分類コード", right_on="level4", how="inner")
    .merge(lv02_df, left_on="経営組織コード", right_on="level2", how="inner")
)

In [ ]:
corp_df["値"].sum()

In [ ]:
corp_df = corp_df.assign(**{
    "雇用者規模": corp_df["企業常用雇用者規模コード"] + "_" + corp_df["企業常用雇用者規模"]
})
org_df = corp_df[["経営組織", "雇用者規模", "値"]]
org_crosstab = org_df.groupby(["経営組織", "雇用者規模"]).sum().reset_index()
org_tidy = org_crosstab.rename(columns={"値": "企業等数"})
# 産業ごとの合計値をグラフ上にテキストとして表示するアノテーション用のデータを用意
org_annotasion = (
    org_tidy[["経営組織", "企業等数"]].groupby("経営組織").sum().astype(int)
)

In [ ]:
algae_colors = plotly.colors.sequential.algae
# Y軸について、最大値の110%を設定して余裕を持たせる
y_max = org_annotasion["企業等数"].max() * 1.2
census_fig = px.bar(
    org_tidy,
    x="経営組織",
    y="企業等数",
    color="雇用者規模",
    color_discrete_sequence=algae_colors,
)
# 経営組織ごとの合計値をグラフ上にテキストとして表示するためScatterを利用
census_fig.add_trace(go.Scatter(
    x=org_annotasion.index,
    y=org_annotasion["企業等数"],
    text=org_annotasion["企業等数"],  # テキストとして表示する内容を設定
    mode="text",  # 描画モードをテキストのみに設定
    textposition="top center",  # テキストの表示位置を各点の上部中央に設定
    textfont={"size": 18},
    texttemplate="%{y:,.0f}社",  # 表示するテキストのフォーマットをカンマ区切り整数として表示
    showlegend=False,  # 凡例を表示しないように設定
))
census_fig.update_layout(width=800, height=500, font={"size": 18},
    yaxis={"tickformat": ",.0f", "ticksuffix": "社", "range": [0, y_max]},
)
census_fig.show()

In [ ]:
cols = ["企業産業中分類階層2", "雇用者規模", "値"]
industry_df = corp_df.loc[corp_df["経営組織"] == "会社企業", cols]
industry_crosstab = (
    industry_df.groupby(["企業産業中分類階層2", "雇用者規模"]).sum().reset_index()
)
industry_tidy = industry_crosstab.rename(
    columns={"企業産業中分類階層2": "産業", "値": "企業等数"}
)
# 産業ごとの合計値をグラフ上にテキストとして表示するアノテーション用のデータを用意
industry_annotasion = (
    industry_tidy[["産業", "企業等数"]].groupby("産業").sum().astype(int)
)

In [ ]:
# 産業ごとの雇用者規模積み上げ企業数の棒グラフを作成
industry_fig = px.bar(
    industry_tidy,
    x="産業",
    y="企業等数",
    color="雇用者規模",
    color_discrete_sequence=algae_colors,
)
# 産業ごとの合計値をグラフ上にテキストとして表示するためScatterを利用
industry_fig.add_trace(go.Scatter(
    x=industry_annotasion.index,
    y=industry_annotasion["企業等数"],
    text=industry_annotasion["企業等数"],  # テキストとして表示する内容を設定
    mode="text",  # 描画モードをテキストのみに設定
    textposition="top center",  # テキストの表示位置を各点の上部中央に設定
    textfont={"size": 10},
    texttemplate="%{y:,.0f}",  # 表示するテキストのフォーマットをカンマ区切り整数として表示
    showlegend=False,  # 凡例を表示しないように設定
))
industry_fig.update_layout(width=1200, height=650, font={"size": 18},
    yaxis={"tickformat": ",.0f", "ticksuffix": "社"},
)
industry_fig.show()

### 産業ツリーマップ


In [ ]:
lvcols = [f"企業産業中分類階層{i}" for i in range(1, 5)]
tree_df = corp_df.loc[corp_df["経営組織"] == "会社企業", lvcols + ["値"]]
tree_crosstab = tree_df.groupby(lvcols).sum().reset_index()
tree_tidy = tree_crosstab.rename(columns={"値": "企業等数"})
tree_tidy.head()

In [ ]:
# Plotly Expressのカラーパレットから「Pastel」と「Set1」を結合して、新しいカラー配列を作成
qualitative_colors = px.colors.qualitative.Set1 + px.colors.qualitative.Pastel
# 産業カテゴリごとにカラーコードを割り当てる辞書を作成
industry_lv2 = tree_tidy["企業産業中分類階層2"].unique()
color_lv2_dict = {
    i: j for i, j in zip(industry_lv2, qualitative_colors[: len(industry_lv2)])
}
# 辞書に未定義のカテゴリ"(?)"があった場合に使う色として配列の次の色を使用
color_lv2_dict.update({"(?)": qualitative_colors[len(industry_lv2)]})
# ツリーマップを用意しツリーマップの最上位の背景色を "lightgrey"に設定
treemap_fig = px.treemap(
    tree_tidy, path=lvcols, values="企業等数", color="企業産業中分類階層2"
)
treemap_fig.update_traces(root_color="lightgrey")
# 各セクションのラベル、値、全体に対する百分率、親セクションに対する百分率のテキスト
# を表示するよう設定
treemap_fig.data[0].textinfo = "label+value+percent root+percent parent"
# ツリーマップの各セクションの色を、先に作成した辞書color_lv2_dictに基づいて設定
treemap_fig.data[0].marker["colors"] = [
    color_lv2_dict[i[0]] for i in treemap_fig.data[0]["customdata"]
]
treemap_fig.show()

## 法人企業統計

- [法人企業統計](https://www.mof.go.jp/pri/reference/ssc/index.htm)

### 業種・資本期規模別の資産合計

- [財務省の業種分類のPDF](https://www.mof.go.jp/pri/reference/ssc/summary/gyosyu.pdf)


In [ ]:
ssc_statsDataId = "0003060791"
ssc_meta = get_metainfo(appId, ssc_statsDataId)
ssc_metadata = ssc_meta["GET_META_INFO"]["METADATA_INF"]
ssc_total_num = ssc_metadata["TABLE_INF"]["OVERALL_TOTAL_NUMBER"]
ssc_total_num

In [ ]:
ssc_cat01 = pd.DataFrame(
    ssc_meta["GET_META_INFO"]["METADATA_INF"]["CLASS_INF"]["CLASS_OBJ"][0]["CLASS"]
)
ssc_cats = [
    "資産合計(当期末)",
    "売上高(当期末)",
    "営業利益(当期末)",
    "経常利益(当期末)",
    "当期純利益(当期末)",
]
ssc_meta_cond = ssc_cat01["@name"].isin(ssc_cats)
ssc_cat01[ssc_meta_cond]

In [ ]:
ssc_levels_str = """level1,level2,level3,level4,level5
104,108,109,109,109\n104,108,110,110,110\n104,108,112,112,112
104,108,113,113,113\n104,108,114,114,114\n104,108,115,115,115
104,108,116,116,116\n104,108,117,117,117\n104,108,118,118,118
104,108,119,119,119\n104,108,120,120,120\n104,108,154,154,154
104,108,121,121,121\n104,108,124,124,124\n104,108,122,122,122
104,108,145,145,145\n104,108,146,123,123\n104,108,146,125,125
104,108,126,126,126\n104,144,105,101,101\n104,144,105,103,103
104,144,106,106,106\n104,144,107,107,107\n104,144,135,135,135
104,144,136,136,136\n104,144,142,142,142\n104,144,134,131,131
104,144,134,132,132\n104,144,134,133,133\n104,144,129,127,127
104,144,129,128,128\n104,144,155,130,130\n104,144,155,149,150
104,144,155,149,151\n104,144,137,156,139\n104,144,137,156,148
104,144,137,157,140\n104,144,137,157,141\n104,144,137,161,138
104,144,137,161,158\n104,144,137,161,159\n104,144,137,153,153
104,144,137,152,152\n104,144,137,160,160\n104,144,137,143,143"""
industry_levels = pd.read_csv(StringIO(ssc_levels_str)).astype(str)

In [ ]:
ssc_cat02 = pd.DataFrame(
    ssc_meta["GET_META_INFO"]["METADATA_INF"]["CLASS_INF"]["CLASS_OBJ"][1]["CLASS"]
)
ssc_cat02 = ssc_cat02.assign(**{"業種": ssc_cat02["@code"] + "_" + ssc_cat02["@name"]})
for lv in industry_levels.columns:
    industry_levels = (industry_levels
        .merge(ssc_cat02[["@code", "業種"]], left_on=lv, right_on="@code", how="left")
        .rename(columns={"業種": "業種" + lv})
        .drop("@code", axis=1)
    )
industry_levels.tail(6)

In [ ]:
ssc_params = {
    "cdCat01": "022",  # 調査項目 (金融業、保険業以外の業種)を資産合計に絞る
    "cdTimeFrom": "20000"  # 2000年以降に絞る
}
asset_data = get_statsdata(appId, ssc_statsDataId, params=ssc_params)
asset_value = colname_to_japanese(cleansing_statsdata(asset_data))
asset_value.columns

In [ ]:
asset_value = asset_value.assign(**{
    "業種": asset_value["業種 (金融業、保険業以外の業種)コード"]
    + "_" + asset_value["業種 (金融業、保険業以外の業種)"],
    "規模": asset_value["規模 (金融業、保険業以外の業種)コード"]
    + "_" + asset_value["規模 (金融業、保険業以外の業種)"],
    "年": asset_value["年  期コード"].str[:4],
})
asset_sizes = [
    "25_10億円以上",
    "24_1億円以上 - 10億円未満",
    "21_5千万円以上 - 1億円未満",
    "20_2千万円以上 - 5千万円未満",
    "17_1千万円以上 - 2千万円未満",
]
ssc_cond = asset_value["規模"].isin(asset_sizes)
ssc_cond &= asset_value["年"] == "2022"
asset_df = asset_value.loc[ssc_cond, ["業種", "規模", "値"]].merge(
    industry_levels, left_on="業種", right_on="業種level5", how="inner"
)
asset_df.head()

In [ ]:
algae_ssc = plotly.colors.sequential.algae[: len(asset_sizes)][::-1]
ssc_fig = px.bar(
    asset_df, x="業種", y="値", color="規模", color_discrete_sequence=algae_ssc
)
ssc_fig.update_layout(width=1200, height=600, font={"size": 14},
    xaxis={"title": None},
    yaxis={"tickformat": ",.0f", "ticksuffix": "百万円", "title": "資産合計"},
    legend={"orientation": "h", "x": -0.05, "y": 1.15, "yanchor": "bottom"},
)
ssc_fig.show()

### 売上高・営業利益・経常利益・当期純利益の推移




In [ ]:
# 売上高のデータ取得
sales_params = {
    "cdCat01": "045",  # 調査項目 (金融業、保険業以外の業種)を売上高に絞る
    "cdCat02": "104",  # 業種 (金融業、保険業以外の業種)を全産業（除く金融保険業）に絞る
    "cdTimeFrom": "20000",  # 2000年以降に絞る
}
sales_data = get_statsdata(appId, ssc_statsDataId, params=sales_params)
sales_value = colname_to_japanese(cleansing_statsdata(sales_data))
# 営業利益のデータ取得
operating_params = {
    "cdCat01": "048",  # 調査項目 (金融業、保険業以外の業種)を営業利益に絞る
    "cdCat02": "104",  # 業種 (金融業、保険業以外の業種)を全産業（除く金融保険業）に絞る
    "cdTimeFrom": "20000",  # 2000年以降に絞る
}
operating_data = get_statsdata(appId, ssc_statsDataId, params=operating_params)
operating_value = colname_to_japanese(cleansing_statsdata(operating_data))
# 経常利益のデータ取得
ordinary_params = {
    "cdCat01": "051",  # 調査項目 (金融業、保険業以外の業種)を経常利益に絞る
    "cdCat02": "104",  # 業種 (金融業、保険業以外の業種)を全産業（除く金融保険業）に絞る
    "cdTimeFrom": "20000",  # 2000年以降に絞る
}
ordinary_data = get_statsdata(appId, ssc_statsDataId, params=ordinary_params)
ordinary_value = colname_to_japanese(cleansing_statsdata(ordinary_data))
# 当期純利益のデータ取得
netincome_params = {
    "cdCat01": "056",  # 調査項目 (金融業、保険業以外の業種)を当期純利益に絞る
    "cdCat02": "104",  # 業種 (金融業、保険業以外の業種)を全産業（除く金融保険業）に絞る
    "cdTimeFrom": "20000",  # 2000年以降に絞る
}
netincome_data = get_statsdata(appId, ssc_statsDataId, params=netincome_params)
netincome_value = colname_to_japanese(cleansing_statsdata(netincome_data))

In [ ]:
ssc_value = pd.concat(
    [sales_value, operating_value, ordinary_value, netincome_value]
).rename(columns={"調査項目 (金融業、保険業以外の業種)": "項目"})
ssc_value = ssc_value.assign(**{
    "規模": ssc_value["規模 (金融業、保険業以外の業種)コード"]
    + "_" + ssc_value["規模 (金融業、保険業以外の業種)"],
    "年": ssc_value["年  期コード"].str[:4],
})
size_cond = ssc_value["規模"].isin(asset_sizes)
ssc_df = ssc_value.loc[size_cond, ["年", "規模", "項目", "値"]]
ssc_df.head()

In [ ]:
pl_cats = [
    "売上高(当期末)",
    "営業利益(当期末)",
    "経常利益(当期末)",
    "当期純利益(当期末)",
]
# 規模の色のリストを作成
algae_mof = plotly.colors.sequential.algae[: len(asset_sizes)][::-1]
# 2*2のサブプロットを持つ図を作成し、各サブプロットにカテゴリ名をタイトルとして設定
subplot_fig = make_subplots(rows=2, cols=2, subplot_titles=pl_cats)
# 各カテゴリごとにサブプロットにプロットする
for i, pl_cat in enumerate(pl_cats):
    row, col = i // 2 + 1, i % 2 + 1  # サブプロットの行と列の位置を計算
    cat_cond = ssc_df["項目"] == pl_cat  # 現在のカテゴリに一致するデータの条件フィルタ
    # 現在のフィルタ条件に一致するデータから「規模」のユニークな値を取得
    unique_sizes = ssc_df.loc[cat_cond, "規模"].unique().tolist()  
    # 現在のフィルタ条件に一致するデータから「年」のユニークな値を取得
    unique_years = ssc_df.loc[cat_cond, "年"].unique()
    # 凡例を一度だけ表示するためのフラグを設定
    legend_added = {s: False for s in unique_sizes}
    # 年ごとのループ
    for year in unique_years:
        # 規模ごとのループ
        for size in unique_sizes:
            # 特定の年、規模、カテゴリに一致するデータの条件フィルタ
            size_cond = (ssc_df["年"] == year) & (ssc_df["規模"] == size) & cat_cond
            # 条件に一致するデータの「値」を合計し、100万で割って兆円単位に変換
            bar_height = ssc_df.loc[size_cond, "値"].sum() / 1_000_000
            # 色のインデックスを規模のリストのインデックスから決定（5色で循環させる）
            color_index = unique_sizes.index(size)
            # 最初のサブプロットで各規模の最初のデータのみ凡例を表示
            show_legend = not legend_added[size] and row == 1 and col == 1
            # 棒グラフのトレースを図に追加
            subplot_fig.add_trace(go.Bar(
                x=[year],  # X軸の値として年を設定
                y=[bar_height],  # Y軸の値として、計算された合計値（兆円単位）を設定
                name=size,  # 各バーの名前として「規模」を設定
                # バーの色を指定されたインデックスの色で設定
                marker={"color": algae_mof[color_index]}, 
                showlegend=show_legend,  # 凡例の表示有無フラグ
            ), row=row, col=col)  # このバーを配置するサブプロットの行と列を指定
            # 凡例を表示した場合、その規模は凡例表示済みとしてフラグを更新
            if show_legend:
                legend_added[size] = True

subplot_fig.update_xaxes(ticksuffix="年")
subplot_fig.update_yaxes(tickformat=",", ticksuffix="兆円")
subplot_fig.update_layout(width=1000, height=800, 
    barmode="stack",
    legend={
        "orientation": "h",
        "x": 0.5,
        "xanchor": "center",
        "y": 1.1,
        "yanchor": "bottom",
    },
)
subplot_fig.show()

## 国税庁の法人番号公表サイト

- [法人番号公表サイト](https://www.houjin-bangou.nta.go.jp/)
- [「基本３情報ダウンロード」のページ](https://www.houjin-bangou.nta.go.jp/download/)
- [法人番号システム Web-API](https://www.houjin-bangou.nta.go.jp/webapi/)
- [適格請求書発行事業者公表サイト](https://www.invoice-kohyo.nta.go.jp/index.html)
- [公表情報ダウンロード](https://www.invoice-kohyo.nta.go.jp/download/index.html)
- [適格請求書発行事業者公表システム Web-API](https://www.invoice-kohyo.nta.go.jp/web-api/index.html)

### CSV形式でデータをダウンロードする


In [ ]:
nta_cols = ["一連番号", "法人番号", "処理区分", "訂正区分", "更新年月日", "変更年月日", 
    "商号又は名称", "商号又は名称イメージID", "法人種別", "国内所在地（都道府県）", 
    "国内所在地（市区町村）", "国内所在地（丁目番地等）", "国内所在地イメージID", 
    "都道府県コード", "市区町村コード", "郵便番号", "国外所在地", "国外所在地イメージID", 
    "登記記録の閉鎖等年月日", "登記記録の閉鎖等の事由", "承継先法人番号", "変更事由の詳細", 
    "法人番号指定年月日", "最新履歴",  "商号又は名称（英語表記）", 
    "国内所在地（都道府県）(英語表記）", "国内所在地（市区町村丁目番地等）（英語表記）", 
    "国外所在地（英語表記）", "フリガナ", "検索対象除外"]

In [ ]:
%%time
nta_ymd = "20250731"  # ここを変更。なおディレクトリ内に複数日付ファイルがなければ空白文字列""も可
csv_files = list(data_path.glob(f"13_tokyo_all_{nta_ymd}*.csv"))
dfs = []
for file in csv_files:
    df = pd.read_csv(file, names=nta_cols, encoding="shift_jis")
    dfs.append(df)
nta_df = pd.concat(dfs)
nta_df.shape

In [ ]:
nta_cond = nta_df["検索対象除外"] == 0
nta_cond &= nta_df["登記記録の閉鎖等年月日"].isna()
houjin_df = nta_df[nta_cond]
houjin_df["法人種別"].value_counts().sort_index()

In [ ]:
kaisha_cond = houjin_df["法人種別"].isin([301, 302, 303, 304, 305])
kaisha_df = houjin_df[kaisha_cond]
kaisha_df["国内所在地（市区町村）"].value_counts().nlargest(20)

### Web-APIでデータを取得する

- [アプリケーションID 発行届出フォーム](https://www.houjin-bangou.nta.go.jp/webapi/#web-api02)


In [ ]:
nta_appid = os.getenv("NTA_APP_ID")  # 取得したアプリケーションIDを指定
houjin_url = "https://api.houjin-bangou.nta.go.jp/4/num"  # バージョン4を利用
houjn_params = {
    "id": nta_appid,
    "number": "6011101063359",
    "type": "02",
}
houjin_res = requests.get(houjin_url, params=houjn_params)
houjin_data_text = houjin_res.text
houjin_data_text

In [ ]:
nta_apidata = pd.read_csv(StringIO(houjin_data_text.split("\n")[1]), names=nta_cols)
nta_apidata.T.head(10)

## 経済産業省のgBizINFO

- [gBizINFO](https://info.gbiz.go.jp/)
- [gBizINFO の出典元](https://info.gbiz.go.jp/resource/index.html)
- [特許情報プラットフォーム](https://www.j-platpat.inpit.go.jp/)
- [職場情報総合サイト](https://shokuba.mhlw.go.jp/)
- [法人活動情報 データダウンロード（情報種別）](https://info.gbiz.go.jp/hojin/DownloadTop)
- [gBizINFO 情報提供REST API](https://info.gbiz.go.jp/api/index.html)
- [次期gBizINFO に関する情報ページ](https://info.gbiz.go.jp/html/R7Infomation.html)

### CSV形式でデータをダウンロードする


In [ ]:
%%time
gbiz_ymd = "20250801"  # ここを変更
gbizdata = pd.read_csv(data_path / f"Kihonjoho_SJIS_{gbiz_ymd}.zip", encoding="shift-jis")

In [ ]:
gbizdata.columns

In [ ]:
int((gbizdata["ステータス"] != "閉鎖").sum())

In [ ]:
gdf = gbizdata.merge(kaisha_df[["法人番号", "商号又は名称"]], on="法人番号", how="inner")
gdf.shape

In [ ]:
int(gdf["従業員数"].count())

In [ ]:
gdf[["法人名", "商号又は名称", "本社所在地", "従業員数"]].nlargest(5, "従業員数")

In [ ]:
gbiz_fig = px.histogram(gdf["従業員数"].apply(np.log10), x="従業員数", nbins=50)
gbiz_fig.update_layout(width=800, height=500, font={"size": 18},
    xaxis_title="従業員数の対数 (log10)",
    yaxis={
        "tickformat": ",.0f",
        "title": "東京の会社数"
    }
)
gbiz_fig.show()

### 【コードのみ】2章のサンプルデータ作成

In [ ]:
sample_cols = ['法人番号', '法人名', '郵便番号', '本社所在地', '資本金', '従業員数', '設立年月日', '創業年', '最終更新日']
gdf = gdf.assign(**{
    '設立年月日': pd.to_datetime(gdf['設立年月日']),
    '最終更新日': pd.to_datetime(gdf['最終更新日']),
})
cond = gdf['設立年月日'] >= "2012-05-18"
cond &= gdf['設立年月日'] <= "2017-09-29"
sample_data = gdf.loc[cond, sample_cols].sort_values("設立年月日").reset_index(drop=True)
#sample_data.to_csv((current_dir / "data" / "ch02").resolve() / "法人データ.csv", index=False, encoding="utf-8")
sample_data

### 【コードのみ】12章のサンプルデータ作成

In [ ]:
usecols = ["法人番号", "法人名", "郵便番号", "本社所在地","ステータス", "設立年月日", "創業年", "最終更新日"]
csv_df = gbizdata[usecols].assign(**{
    "設立年月日": lambda df: pd.to_datetime(df["設立年月日"]),
    "最終更新日": lambda df: pd.to_datetime(df["最終更新日"])
})
trans_map = {
    "法人番号": "corporate_number", "法人名": "name",
    "本社所在地": "location", "郵便番号": "postal_code",
    "ステータス": "status", "最終更新日": "update_date"
}
required_cols = list(trans_map.keys()) + ["founded_year"]
founded_year_combined = csv_df["設立年月日"].dt.year.fillna(csv_df["創業年"]).astype("Int64")
founded_from_2019 = (csv_df["設立年月日"].dt.year >= 2019) | (csv_df["創業年"] >= 2019)
founded_until_2023 = (csv_df["設立年月日"].dt.year < 2024) | (csv_df["創業年"] < 2024)
gbiz_merged_df = (gbizdata
    .assign(**{"founded_year": founded_year_combined})
    .loc[csv_df["法人名"].notna() &
        founded_from_2019 & founded_until_2023 &
        (founded_year_combined.isna() | (founded_year_combined >= 2019)),
        required_cols]
    .rename(columns=trans_map)
)
#gbiz_merged_df.to_csv(data_path / "gbiz_startup_df.csv", index=False)
gbiz_merged_df.shape

### REST APIを利用する

- [gBizINFO REST API 仕様書](https://info.gbiz.go.jp/hojin/swagger-ui/index.html#/gBizINFO%20REST%20API/searchInfo)


In [ ]:
headers_gbiz = {"Accept": "application/json", "X-hojinInfo-api-token": api_token}
url_hojin = "https://info.gbiz.go.jp/hojin/v1/hojin"
params_hojin = {
    "corporate_type": "301",  # 株式会社
    "prefecture": "13",  # 東京
    "founded_year": ",".join(str(y) for y in range(2010, 2025)),  # 創業年
    "year": "2023",  # 年度
    "limit": "5000",  # デフォルトは1000
}
res_hojin = requests.get(url_hojin, params=params_hojin, headers=headers_gbiz)
hojin_json = res_hojin.json()
hojin_json.keys()

In [ ]:
hojin_infos = pd.json_normalize(hojin_json, record_path=["hojin-infos"])
hojin_infos.columns

In [ ]:
hojin_infos.head()